<a href="https://colab.research.google.com/github/chun1979/Big-roof_Expo/blob/main/Big_roof_Expo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

以下，大屋根をJSON, PLYから再現するためのコードです．Google Colabでの動作を前提としたコードなので，ローカル環境で活用する場合は適宜改変をお願いいたします．
また，ファイルパスなどもご自身の環境に沿って改変してください．

In [ ]:
#Google Driveに接続するため．この部分はローカルで動かす場合は不要．
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Google Colabに入っていないパッケージのインストール．ローカルの場合は普通にpipなどでパッケージをインストールしてください．
!pip install plyfile

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import csv

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
from matplotlib.colors import Normalize

import plyfile
from collections import defaultdict
from scipy.spatial import Delaunay

import os
from plyfile import PlyData, PlyElement, PlyListProperty
import matplotlib.patches as patches
from shapely.geometry import Polygon
import math
import json
%cd "/content/drive/My Drive/Touden_Banpaku/temp2"
%ls
#対象フォルダに移動してください．

In [9]:
#柱を配置していくパート
def get_structure_data(json_file, target_id, target_structure_type):
    # JSONファイルを読み込む
    with open(json_file, 'r') as file:
        structures = json.load(file)

    # 指定されたIDとstructure_typeに一致するデータを検索
    for structure in structures:
        if structure["ID"] == target_id and structure["structure_type"] == target_structure_type:
            return {
                "p1": structure["coordinates"]["p1"],
                "p2": structure["coordinates"]["p2"],
                "p3": structure["coordinates"]["p3"],
                "p4": structure["coordinates"]["p4"],
                "roof_url": structure["structure_url"]["roof_url"]
                #"structure_url": structure["structure_url"]
            }

    # 該当するデータが見つからない場合
    return None

def interpolate(p1, p2, t):
    """ 線分上の点を線形補間で求める """
    return p1[0] + (p2[0] - p1[0]) * t, p1[1] + (p2[1] - p1[1]) * t

def create_ply_file(points_groups, filename):
    # ヘッダー部分の準備
    header = [
        "ply",
        "format ascii 1.0"
    ]

    # 各グループから頂点データと面データを生成
    vertex_data = []
    face_data = []
    vertex_count = 0  # 頂点の累積数

    for absolute_points in points_groups:
        # 現在のグループの頂点データを追加
        vertex_data.extend([f"{x} {y} {z}" for x, y, z in absolute_points])

        # 面データの生成（頂点インデックスは累積された頂点数を考慮）
        num_faces = len(absolute_points) // 2 - 1
        face_data.extend([f"4 {vertex_count + 2 * i} {vertex_count + 2 * i + 1} {vertex_count + 2 * i + 3} {vertex_count + 2 * i + 2}" for i in range(num_faces)])

        # 頂点数を更新
        vertex_count += len(absolute_points)

    # ヘッダーに頂点と面の要素数を追加
    header.append(f"element vertex {vertex_count}")
    header.append("property float x")
    header.append("property float y")
    header.append("property float z")
    header.append(f"element face {len(face_data)}")
    header.append("property list uchar int vertex_indices")
    header.append("end_header")

    # ファイルに書き込む
    with open(filename, 'w') as file:
        file.write('\n'.join(header) + '\n')
        file.write('\n'.join(vertex_data) + '\n')
        file.write('\n'.join(face_data) + '\n')

# 新しいJSONファイルからデータを読み込んでPLYファイルを生成する関数
def create_ply_from_json(json_file, ply_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
        points_group = [[(point['x_relative'], point['y_relative'], point['z']) for point in data["roof1"]]]

    create_ply_file(points_group, ply_file)

def read_p1_p2_pairs_and_template_files_from_json_filtered(json_file):
    """ Read p1, p2 pairs and corresponding template files from a JSON file, filtering for 'roof and pillars' structure type. """
    with open(json_file, 'r') as file:
        data = json.load(file)

    p1_p2_pairs_and_templates = []
    for item in data:
        if item['structure_type'] == 'roof and pillars':  # 'roof and pillars'タイプのみをフィルタリング
            p1 = item['coordinates']['p1']
            p2 = item['coordinates']['p2']
            template_file = item['structure_url']['pillar_url']
            p1_p2_pairs_and_templates.append((p1, p2, template_file))

    return p1_p2_pairs_and_templates



def read_ply(filename):
    """ Read a PLY file and extract vertices and faces. """
    with open(filename, 'r') as file:
        content = file.readlines()

    # Find the header end
    end_header_index = 0
    for i, line in enumerate(content):
        if "end_header" in line:
            end_header_index = i
            break

    # Extract vertices and faces
    vertices = []
    faces = []
    for line in content[end_header_index+1:]:
        if line.strip():
            parts = line.split()
            if len(parts) == 3:
                # Vertex line
                vertices.append([float(coord) for coord in parts])
            elif len(parts) > 3:
                # Face line
                faces.append([int(index) for index in parts[1:]])

    return np.array(vertices), faces

def rotate_and_translate(vertices, angle, translation):
    """ Rotate vertices by a given angle and then translate. """
    # Rotation matrix
    rotation_matrix = np.array([
        [math.cos(angle), -math.sin(angle)],
        [math.sin(angle),  math.cos(angle)]
    ])

    # Apply rotation and translation
    rotated_vertices = np.dot(vertices[:, :2], rotation_matrix.T) + translation

    # Update the vertices with the new values
    vertices[:, :2] = rotated_vertices

    return vertices

def save_ply(filename, vertices, faces):
    """ Save the vertices and faces to a new PLY file. """
    with open(filename, 'w') as file:
        # Write PLY header
        file.write("ply\n")
        file.write("format ascii 1.0\n")
        file.write(f"element vertex {len(vertices)}\n")
        file.write("property float x\n")
        file.write("property float y\n")
        file.write("property float z\n")
        file.write(f"element face {len(faces)}\n")
        file.write("property list uchar int vertex_indices\n")
        file.write("end_header\n")

        # Write vertex data
        for vertex in vertices:
            file.write(f"{vertex[0]} {vertex[1]} {vertex[2]}\n")

        # Write face data
        for face in faces:
            file.write(f"{len(face)} {' '.join(map(str, face))}\n")


def process_and_combine_buildings(p1_p2_pairs_and_templates, output_file):
    """ Process multiple buildings with different templates and combine them into a single PLY file. """
    combined_vertices = []
    combined_faces = []
    vertex_count = 0
    point_before_translation = [0, 5]

    for p1, p2, template_file in p1_p2_pairs_and_templates:
        # Process each building
        vertices, faces = read_ply(template_file)
        adjusted_angle = calculate_rotation_angle(p1, p2, point_before_translation)

        # Rotate and translate vertices
        transformed_vertices = rotate_and_translate(vertices, adjusted_angle, p1)
        combined_vertices.extend(transformed_vertices)

        # Adjust face indices and add to combined list
        adjusted_faces = [[vertex_count + index for index in face] for face in faces]
        combined_faces.extend(adjusted_faces)

        # Update the vertex count for the next building
        vertex_count += len(vertices)

    # Save the combined building data to a new PLY file
    save_ply(output_file, np.array(combined_vertices), combined_faces)

def calculate_rotation_angle(p1, p2, point_after_translation):
    """
    p1 の位置に移動した後、point_after_translation が p1 と p2 の間の線上に来るように回転させる角度を計算する関数
    :param p1: 平行移動後の原点の位置
    :param p2: 直線の第二点
    :param point_after_translation: 平行移動後の点 (0, 6.336)
    :return: 回転角度（ラジアン）
    """
    # p1 から p2 へのベクトル
    direction_vector = np.array(p2) - np.array(p1)

    # 平行移動後の点の方向ベクトル
    point_vector = np.array(point_after_translation) - np.array([0,0])#np.array(p1)

    # 両方のベクトルの間の角度を計算
    angle = np.arctan2(direction_vector[1], direction_vector[0]) - np.arctan2(point_vector[1], point_vector[0])
    return angle

# 使用例
json_file = 'structures3.json'
# 結果の出力
p1_p2_pairs_and_templates = read_p1_p2_pairs_and_template_files_from_json_filtered(json_file)
output_file = "combined_buildings_x4.ply"
process_and_combine_buildings(p1_p2_pairs_and_templates, output_file)




In [ ]:
absolute_points_list = []
#屋根を配置していくパート
#pillarがない場所の屋根
target_structure_type = "roof"
for target_id in range(109):
    #print(target_id)
    structure_data = get_structure_data(json_file, target_id, target_structure_type)

    # 新しいJSONファイル形式に基づいて点のデータを読み込む
    if structure_data["roof_url"] == "./structure_details/roofonly_f1.json":
        with open(structure_data["roof_url"], 'r') as file:
            data = json.load(file)
            points_data = data["roof1"]  # "roof1" キーの下のデータを取得

        # p1, p2, p3, p4の座標を取得
        print(structure_data)
        p1, p2, p3, p4 = structure_data["p1"], structure_data["p2"], structure_data["p3"], structure_data["p4"]

        # 絶対座標の計算
        absolute_points = []
        for point in points_data:
            x_rel, y_rel, z = point["x_relative"], point["y_relative"], point["z"]
            x1, y1 = interpolate(p1, p2, y_rel)  # p1とp4間のy軸補間
            x2, y2 = interpolate(p4, p3, y_rel)  # p2とp3間のy軸補間
            x = interpolate((x1, y1), (x2, y2), x_rel)[0]  # x軸補間で最終的なx座標
            y = interpolate((x1, y1), (x2, y2), x_rel)[1]  # x軸補間で最終的なy座標
            absolute_points.append((x, y, z))

        absolute_points_list.append(absolute_points)

    elif structure_data["roof_url"] == "./structure_details/roofonly_f2.json":
        with open(structure_data["roof_url"], 'r') as file:
            data = json.load(file)
            points_data1 = data["roof1"]  # "roof1" キーの下のデータを取得
            points_data2 = data["roof2"]  # "roof1" キーの下のデータを取得

        # p1, p2, p3, p4の座標を取得
        print(structure_data)
        p1, p2, p3, p4 = structure_data["p1"], structure_data["p2"], structure_data["p3"], structure_data["p4"]

        # 絶対座標の計算
        absolute_points = []
        for point in points_data1:
            x_rel, y_rel, z = point["x_relative"], point["y_relative"], point["z"]
            x1, y1 = interpolate(p1, p2, y_rel)  # p1とp4間のy軸補間
            x2, y2 = interpolate(p4, p3, y_rel)  # p2とp3間のy軸補間
            x = interpolate((x1, y1), (x2, y2), x_rel)[0]  # x軸補間で最終的なx座標
            y = interpolate((x1, y1), (x2, y2), x_rel)[1]  # x軸補間で最終的なy座標
            absolute_points.append((x, y, z))
        absolute_points_list.append(absolute_points)

        absolute_points = []
        for point in points_data2:
            x_rel, y_rel, z = point["x_relative"], point["y_relative"], point["z"]
            x1, y1 = interpolate(p1, p2, y_rel)  # p1とp4間のy軸補間
            x2, y2 = interpolate(p4, p3, y_rel)  # p2とp3間のy軸補間
            x = interpolate((x1, y1), (x2, y2), x_rel)[0]  # x軸補間で最終的なx座標
            y = interpolate((x1, y1), (x2, y2), x_rel)[1]  # x軸補間で最終的なy座標
            absolute_points.append((x, y, z))
        absolute_points_list.append(absolute_points)

#pillarがある場所の屋根
target_structure_type = "roof and pillars"
for target_id in range(109):
    #print(target_id)
    structure_data = get_structure_data(json_file, target_id, target_structure_type)

    # 新しいJSONファイル形式に基づいて点のデータを読み込む
    if structure_data["roof_url"] == "./structure_details/roofonly_f1.json":
        with open(structure_data["roof_url"], 'r') as file:
            data = json.load(file)
            points_data = data["roof1"]  # "roof1" キーの下のデータを取得

        # p1, p2, p3, p4の座標を取得
        print(structure_data)
        p1, p2, p3, p4 = structure_data["p1"], structure_data["p2"], structure_data["p3"], structure_data["p4"]

        # 絶対座標の計算
        absolute_points = []
        for point in points_data:
            x_rel, y_rel, z = point["x_relative"], point["y_relative"], point["z"]
            x1, y1 = interpolate(p1, p2, y_rel)  # p1とp4間のy軸補間
            x2, y2 = interpolate(p4, p3, y_rel)  # p2とp3間のy軸補間
            x = interpolate((x1, y1), (x2, y2), x_rel)[0]  # x軸補間で最終的なx座標
            y = interpolate((x1, y1), (x2, y2), x_rel)[1]  # x軸補間で最終的なy座標
            absolute_points.append((x, y, z))

        absolute_points_list.append(absolute_points)

    elif structure_data["roof_url"] == "./structure_details/roofonly_f2.json":
        with open(structure_data["roof_url"], 'r') as file:
            data = json.load(file)
            points_data1 = data["roof1"]  # "roof1" キーの下のデータを取得
            points_data2 = data["roof2"]  # "roof1" キーの下のデータを取得

        # p1, p2, p3, p4の座標を取得
        print(structure_data)
        p1, p2, p3, p4 = structure_data["p1"], structure_data["p2"], structure_data["p3"], structure_data["p4"]

        # 絶対座標の計算
        absolute_points = []
        for point in points_data1:
            x_rel, y_rel, z = point["x_relative"], point["y_relative"], point["z"]
            x1, y1 = interpolate(p1, p2, y_rel)  # p1とp4間のy軸補間
            x2, y2 = interpolate(p4, p3, y_rel)  # p2とp3間のy軸補間
            x = interpolate((x1, y1), (x2, y2), x_rel)[0]  # x軸補間で最終的なx座標
            y = interpolate((x1, y1), (x2, y2), x_rel)[1]  # x軸補間で最終的なy座標
            absolute_points.append((x, y, z))
        absolute_points_list.append(absolute_points)

        absolute_points = []
        for point in points_data2:
            x_rel, y_rel, z = point["x_relative"], point["y_relative"], point["z"]
            x1, y1 = interpolate(p1, p2, y_rel)  # p1とp4間のy軸補間
            x2, y2 = interpolate(p4, p3, y_rel)  # p2とp3間のy軸補間
            x = interpolate((x1, y1), (x2, y2), x_rel)[0]  # x軸補間で最終的なx座標
            y = interpolate((x1, y1), (x2, y2), x_rel)[1]  # x軸補間で最終的なy座標
            absolute_points.append((x, y, z))
        absolute_points_list.append(absolute_points)

filename = 'output_roofonly_f1f2_x5.ply'
create_ply_file(absolute_points_list, filename)
